# NGC Catalogのコンテナを実行する--TensorFlow-Apptainer

---

構築したOpenHPC環境でNGC Catalogの[TensorFlowコンテナ](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow)を実行します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる
* SlurmのGeneric Resource(GRES)としてGPUが登録されていること


マスターノードに対して SSH でログインできることを確認します。マスターノードのIPアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address = 

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'
# user = 'user00'

user = 

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

SlurmのGeneric Resource(GRES)としてGPUが登録されていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo -N --Format=NodeHost,Gres | grep -w gpu

## TensorFlowコンテナの実行

TensorFlowコンテナでMNISTを実行してみます。

### コンテナイメージの取得

OpenHPC環境でコンテナを実行するにはApptainerを利用します。はじめにNGCカタログから[TensorFlow](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow)のコンテナイメージを取得します。

> 作業領域として `/tmp` に 30GB程度の空き領域が必要となります。他のディレクトリを作業領域として利用する場合は環境変数`APPTAINER_TMPDIR`を指定してください。詳細についてはApptainerのドキュメント [Build Customization](https://apptainer.org/docs/user/1.3/build_env.html#temporary-folders) を参照してください。
> 
> またイメージの取得、変換には６０分程度かかります。

In [ ]:
ngc_version = '25.02'
sif_file = f'tensorflow_{ngc_version}-tf2-py3.sif'

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'test -f {sif_file} || apptainer pull docker://nvcr.io/nvidia/tensorflow:{ngc_version}-tf2-py3'"

### 準備

MNISTのデータやスクリプトを準備します。

まず、データやスクリプトを配置するディレクトリを作成します。

In [ ]:
work_dir = 'tensorflow'
!ssh {ssh_opts} {target} mkdir -p {work_dir}

データをダウンロードするためのスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/download_mnist.py {target}:{work_dir}

スクリプトを実行し、データをダウンロードします。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && apptainer exec ~/{sif_file} python download_mnist.py'"

データが取得できたことを確認します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'ls -lR ~/.keras/datasets'"

TensorFlowの[Quickstart for beginners](https://www.tensorflow.org/tutorials/quickstart/beginner)で実行しているのと同等のスクリプトをOpenHPC環境に配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/mnist_classify.py {target}:{work_dir}

取得したスクリプトの内容を先頭部分を表示してみます。

In [ ]:
!ssh {ssh_opts} {target} head {work_dir}/mnist_classify.py

### コンテナからGPUが利用できることを確認する

Apptainerで実行したコンテナ環境からGPUを利用できることを確認します。

GPUを利用できるかをチェックするスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/check_gpu.py {target}:{work_dir}

GPU利用の可否をチェックするスクリプトを実行します。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && srun -l -N 1 \
    apptainer exec --nv ~/{sif_file} python check_gpu.py'"

### ジョブの実行

MNISTのスクリプトをSlurmのジョブとして実行します。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'tensorflow_mnist.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J tensorflow-mnist        # create a short name for your job
#SBATCH -o tensorflow-mnist.%j.out # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                       # Total number of nodes requested
#SBATCH -n 1                       # Total number of across all nodes
#SBATCH --gres=gpu:1               # number of gpus per node
#SBATCH -t 00:10:00                # Run time (hh:mm:ss)

cd $HOME/{work_dir}
apptainer exec --nv $HOME/{sif_file} python3 mnist_classify.py
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行する前のキューの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブを実行します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch tensorflow_mnist.job'"

ジョブの実行状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。`GRES_USED` の欄でノードのGPU利用状況を確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブが完了するまで数分かかります。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することができます。

ジョブの完了後に次のセルを実行してください。ジョブの出力結果が確認できます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```